In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import load_model, Model

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [4]:
data = pd.read_excel('dataBerita(augmentasi).xlsx', usecols=['kategori', 'berita'])

In [5]:
data.head()

,kategori,berita
0,valid;,Gunung Agung erupsi untuk pertama kali pada 2...
1,valid;,Jakarta CNN Indonesia -- Menteri BUMN Erick T...
2,valid;,Dosen Fakultas Kedokteran Hewan IPB Yusuf Ridw...
3,valid;,Jakarta - Dua anggota TNI Serda N dan Serda DA...
4,valid;,Akui Tembak Jatuh Pesawat Ukraina Iran Tuai Ge...


In [6]:
print(data.columns)


Index(['kategori', 'berita'], dtype='object')


In [7]:
print(data['kategori'].unique())

['valid;' '\nvalid;' 'valid; ' 'hoax;']


In [8]:
# Membersihkan data dengan menghapus whitespace dan karakter tidak perlu
data['kategori'] = data['kategori'].str.strip()        # Menghapus whitespace di awal dan akhir
data['kategori'] = data['kategori'].str.replace(';', '', regex=False)  # Menghapus tanda semicolon

In [9]:
print(data['kategori'].unique())

['valid' 'hoax']


In [10]:
label_counts = data['kategori'].value_counts()

print("Number of hoax samples:", label_counts.get('hoax', 0))
print("Number of valid samples:", label_counts.get('valid', 0))

Number of hoax samples: 432
Number of valid samples: 432


In [11]:
data['kategori_enc'] = data['kategori'].map({'valid': 0, 'hoax': 1})

In [12]:
data.head()

,kategori,berita,kategori_enc
0,valid,Gunung Agung erupsi untuk pertama kali pada 2...,0
1,valid,Jakarta CNN Indonesia -- Menteri BUMN Erick T...,0
2,valid,Dosen Fakultas Kedokteran Hewan IPB Yusuf Ridw...,0
3,valid,Jakarta - Dua anggota TNI Serda N dan Serda DA...,0
4,valid,Akui Tembak Jatuh Pesawat Ukraina Iran Tuai Ge...,0


In [13]:
x = data.drop('kategori_enc',axis=1)
y = data['kategori_enc']

Data Pre-processing and One Hot Represenatation

In [14]:
from collections import Counter
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    return words

def analyze_word_frequency(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Preprocess and count word frequencies
    words = preprocess_text(text)
    word_freq = Counter(words)
    
    # Sort frequencies in descending order
    sorted_frequencies = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    
    print("Total unique words:", len(word_freq))
    print("\nTop 20 most frequent words:")
    for word, freq in sorted_frequencies[:20]:
        print(f"{word}: {freq}")
    
    return word_freq, sorted_frequencies

# Usage
file_path = 'corpus_Aug.txt'
word_frequencies, sorted_freq = analyze_word_frequency(file_path)

Total unique words: 6641

Top 20 most frequent words:
orang: 460
indonesia: 341
presiden: 239
jakarta: 237
milik: 198
anak: 197
rumah: 196
polisi: 195
korban: 186
negara: 183
salah: 171
jalan: 167
hasil: 166
bom: 161
temu: 158
masyarakat: 157
perintah: 156
tinggal: 156
serang: 155
lapor: 153


In [16]:
# Read the corpus from the text file
corpus = []
with open('corpus_Aug.txt', 'r', encoding='utf-8') as f:
    corpus = [line.strip() for line in f.readlines()]


In [17]:
# Hitung panjang setiap kalimat
sentence_lengths = [len(sentence.split()) for sentence in corpus]

# Temukan panjang maksimum dan kalimat yang memiliki panjang maksimum tersebut
max_length = max(sentence_lengths)
max_length_index = sentence_lengths.index(max_length)
longest_sentence = corpus[max_length_index]

# Menghitung panjang setiap kalimat
sentence_lengths = [len(sentence.split()) for sentence in corpus]

# Menemukan panjang pada persentil ke-95
maxlen_recommended = int(np.percentile(sentence_lengths, 95))

print("Panjang kalimat maksimum:", max_length)
print("Kalimat dengan panjang maksimum:", longest_sentence)
print("Panjang maxlen yang direkomendasikan (95 persentil):", maxlen_recommended)

Panjang kalimat maksimum: 625
Kalimat dengan panjang maksimum: liput com jakarta desah kokpit maskapai terbang lion air marak berita tanah air dunia sorot kutip news com au sabtu media australia tulis picu maskapai terbang murah besar indonesia lidi internal kopilot larang terbang tulis media negeri kanguru kutip nyata direktur lion air edward sirait nyata langkah cabut izin terbang kompensasi si kopilot aku cipta desah heboh kejut ulang muat media inggris daily mail situs berita tajuk artikel kait pilot banned from flying after offering divorced air hostess as compensation for a delay on a flight to bal media pakistan tribune pk artikel indonesian pilot grounded for offering hostess as compensation berita heboh lapor tumpang nama lambertus maengkom media pakistan beber pesawat lion air lepas landas lambat pulau jawa kopilot canda tawar pramugari janda tugas minta maaf keras suara situs singapura the straits times tinggal tulis desah kokpit lion air laman judul indonesia s lion air gro

In [55]:
# We need to reset the indices as we have sampled our initial news dataset
msg = x.copy()
msg.reset_index(inplace=True)

In [13]:
# import StemmerFactory class
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
#Data pre-pocessing

corpus=[]
for i in range(0,len(msg)):
    review= re.sub('[^a-zA-Z]',' ',msg['berita'][i])
    review= review.lower()
    review= review.split()

    review= [stemmer.stem(word) for word in review if not word in stopwords.words('indonesian')]
    review= ' '.join(review)
    corpus.append(review)

In [69]:
# Read the corpus from the text file
corpus = []
with open('corpus.txt', 'r', encoding='utf-8') as f:
    corpus = [line.strip() for line in f.readlines()]


In [18]:
len(corpus)

864

In [95]:
#one-hot frepresentation
voc_size = 5500
one_hot_re = [one_hot(element,voc_size) for element in corpus]

#using pad_sequences to make all the representations of fixed length
sentence_length=171
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

In [88]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus)

# Pad the sequences to ensure the same length as the training data
sentence_length = 171  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

Creating Model

In [96]:
x_final = np.array(embedded_docs)
y_final= np.array(y)
x_train,x_test, y_train,y_test = train_test_split(x_final, y_final,test_size=0.33,random_state=42,stratify=y_final)

In [21]:
# Set up k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

### LSTM

In [90]:
embedding_vector_features = 100
voc_size=6500
sentence_length=171
print(voc_size)
print(embedding_vector_features)
print(sentence_length)

6500
100
171


In [91]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    lstm_layer = LSTM(32)(embedding_layer)
    outputs = Dense(1, activation='sigmoid')(lstm_layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [92]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=171)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-lstm.keras')


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Best Model Accuracy: 0.976878612716763
Best Model Precision: 0.9743589743589743
Best Model Recall: 0.9743589743589743
Best Model F1-Score: 0.9743589743589743
Best Model Validation Loss: [0.6327808499336243, 0.4997037351131439, 0.35079246759414673, 0.23313888907432556, 0.17349664866924286, 0.11576905101537704, 0.09121519327163696, 0.07915609329938889, 0.07426758855581284, 0.07154098898172379]
Best Model Validation Accuracy: [0.8901734352111816, 0.849711000919342, 0.9190751314163208, 0.9537572264671326, 0.9595375657081604, 0.9710982441902161, 0.9826589822769165, 0.9826589822769165, 0.9826589822769165, 0.9768785834312439]


In [93]:
# Load the best model
best_model = load_model('t-100-6500-lstm.keras')

In [30]:
# Test the model with new data
test = pd.read_excel('../test.xlsx', usecols=['kategori', 'berita'])

test.head()

,berita,kategori
0,Hakim Wahyu Iman Santoso Alami Kecelakaan Tung...,1
1,MEGAWATI DAN PUAN BERMAIN SLOT Nenek lampir pe...,1
2,JONATHAN LATUMAHINA SEORANG NASRANI DAN PENYUS...,1
3,"PDI-P Diblacklist dari Peserta Pilpres, Tak Bi...",1
4,Presiden Joe Biden dan Volodymyr Zelenskyy Ber...,1


In [31]:
print(test.shape)
print(test['kategori'].unique())

(1000, 2)
[1 0]


In [32]:
# Count occurrences of each label in the 'kategori' column
label_counts = test['kategori'].value_counts()

# Display the counts for "hoax" and "valid" (adjust if labels are different)
print("Number of hoax samples:", label_counts.get(1, 0))
print("Number of valid samples:", label_counts.get(0, 0))

Number of hoax samples: 500
Number of valid samples: 500


In [34]:
# Encode the target labels (0 for valid, 1 for hoax)
test['kategori_enc'] = test['kategori']

In [68]:
# Initialize the Sastrawi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Preprocess text: cleaning, stemming, removing stopwords
corpus_test = []
for i in range(len(test)):
    review = re.sub('[^a-zA-Z]', ' ', test['berita'][i])  # Remove non-letter characters
    review = review.lower()
    review = review.split()
    review = [stemmer.stem(word) for word in review if word not in stopwords.words('indonesian')]
    review = ' '.join(review)
    corpus_test.append(review)

In [74]:
# Save the `corpus` to a text file
with open('corpus_test.txt', 'w') as file:
    for sentence in corpus_test:
        file.write(sentence + '\n')

print("Corpus saved as 'corpus_test.txt'")

Corpus saved as 'corpus_test.txt'


In [36]:
len(corpus_test)

1000

In [35]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

In [72]:
# Define necessary preprocessing parameters
voc_size = 5500           # Must match training
sentence_length = 171   # Must match training
print(voc_size)
print(sentence_length)

5500
171


In [65]:
# Convert text to sequences using one-hot encoding and pad the sequences
one_hot_re = [one_hot(sentence, voc_size) for sentence in corpus_test]
new_x_final = pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values
# Predict with the new data
new_predictions = (best_model.predict(new_x_final) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Accuracy on new data: 0.725
Precision on new data: 0.7764127764127764
Recall on new data: 0.632
F1 Score on new data: 0.6968026460859978


In [94]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 171  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Accuracy on new data: 0.729
Precision on new data: 0.865814696485623
Recall on new data: 0.542
F1 Score on new data: 0.6666666666666666


### Using Dropout Layer

In [214]:
# Read the corpus from the text file
corpus = []
with open('corpus_Aug.txt', 'r', encoding='utf-8') as f:
    corpus = [line.strip() for line in f.readlines()]

len(corpus)

In [153]:
voc_size=6500
embedding_vector_features=100

print(voc_size)
print(embedding_vector_features)
print(sentence_length)

6500
100
171


In [125]:
#one-hot frepresentation

one_hot_re = [one_hot(element,voc_size) for element in corpus]

#using pad_sequences to make all the representations of fixed length
sentence_length=171
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

In [154]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus)

# Pad the sequences to ensure the same length as the training data
sentence_length = 171  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

In [155]:
x_final = np.array(embedded_docs)
y_final= np.array(y)
x_train,x_test, y_train,y_test = train_test_split(x_final, y_final,test_size=0.33,random_state=42,stratify=y_final)

In [156]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    dropout_layer_1 = Dropout(0.2)(embedding_layer)

    lstm_layer = LSTM(32)(dropout_layer_1)
    dropout_layer_2 = Dropout(0.2)(lstm_layer)

    outputs = Dense(1, activation='sigmoid')(dropout_layer_2)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [157]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=171)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-lstm_w_d.keras')


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Best Model Accuracy: 0.9826589595375722
Best Model Precision: 0.9746835443037974
Best Model Recall: 0.9871794871794872
Best Model F1-Score: 0.9808917197452229
Best Model Validation Loss: [0.6299266219139099, 0.480945348739624, 0.328502357006073, 0.2132781744003296, 0.15389706194400787, 0.1300736665725708, 0.11317156255245209, 0.09638959169387817, 0.07091765105724335, 0.06988024711608887]
Best Model Validation Accuracy: [0.9190751314163208, 0.9017341136932373, 0.9248554706573486, 0.9421965479850769, 0.9479768872261047, 0.9653179049491882, 0.9653179049491882, 0.9710982441902161, 0.9826589822769165, 0.9826589822769165]


In [158]:
# Load the best model 828387478
best_model = load_model('t-100-6500-lstm_w_d.keras')

In [76]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

len(corpus_test)

1000

In [258]:
len(corpus_test)

1000

In [114]:
print(voc_size)

6500


In [159]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 171  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
Accuracy on new data: 0.743
Precision on new data: 0.8461538461538461
Recall on new data: 0.594
F1 Score on new data: 0.6980023501762632


In [130]:
voc_size=6500
sentence_length=171

# Convert text to sequences using one-hot encoding and pad the sequences
one_hot_re = [one_hot(sentence, voc_size) for sentence in corpus_test]
new_x_final = pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

# Prepare the labels
new_y_final = test['kategori_enc'].values

# Predict with the new data
new_predictions = (best_model.predict(new_x_final) > 0.5).astype("int32").flatten()

# Evaluate the new predictions
new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Accuracy on new data: 0.72
Precision on new data: 0.7370689655172413
Recall on new data: 0.684
F1 Score on new data: 0.7095435684647303


### Bidirectional LSTM

In [248]:
# Read the corpus from the text file
corpus = []
with open('corpus.txt', 'r', encoding='utf-8') as f:
    corpus = [line.strip() for line in f.readlines()]

In [217]:
voc_size = 6500
embedding_vector_features = 100
print(len(corpus))
print(voc_size)
print(sentence_length)

864
6500
171


In [187]:
#one-hot frepresentation

one_hot_re = [one_hot(element,voc_size) for element in corpus]

#using pad_sequences to make all the representations of fixed length
sentence_length=171
embedded_docs= pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

In [218]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus)

# Pad the sequences to ensure the same length as the training data
sentence_length = 171  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

In [219]:
x_final = np.array(embedded_docs)
y_final= np.array(y)
x_train,x_test, y_train,y_test = train_test_split(x_final, y_final,test_size=0.33,random_state=42,stratify=y_final)

In [220]:
def create_model(voc_size, embedding_vector_features, sentence_length):
    inputs = tf.keras.Input(shape=(sentence_length,))
    embedding_layer = Embedding(voc_size, embedding_vector_features)(inputs)
    bi_layer = Bidirectional(LSTM(32))(embedding_layer)

    outputs = Dense(1, activation='sigmoid')(bi_layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [221]:
best_accuracy = 0 
best_precision = 0 
best_recall = 0  
best_f1 = 0

best_model = None 

best_val_loss = None  
best_val_accuracy = None  

for train_index, test_index in kf.split(x_final):
    x_train, x_test = x_final[train_index], x_final[test_index]
    y_train, y_test = y_final[train_index], y_final[test_index]
    
    # Create a new model for each fold
    model = create_model(voc_size=6500, embedding_vector_features=100, sentence_length=171)
    
    # Set up the model checkpoint to save the best model during training
    checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=0)
    tensorboard_callback = TensorBoard(log_dir='./logs')

    # Train the model on the current fold with validation data
    history = model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpoint, tensorboard_callback])  
    
    # Load the best model based on validation loss
    model = load_model('best_model.keras')

    # Predict on the test set
    y_pred = (model.predict(x_test) > 0.5).astype("int32").flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Track the best model based on accuracy
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_precision = precision
        best_recall = recall
        best_f1 = f1

        best_model = model
        # Store the validation loss and accuracy for the best model
        best_val_loss = history.history['val_loss']
        best_val_accuracy = history.history['val_accuracy']
    
print("Best Model Accuracy:", best_accuracy)
print("Best Model Precision:", best_precision)
print("Best Model Recall:", best_recall)
print("Best Model F1-Score:", best_f1)

# After the K-fold cross-validation loop, print eval losses and accuracies for the best model
print("Best Model Validation Loss:", best_val_loss)
print("Best Model Validation Accuracy:", best_val_accuracy)

# Optionally, save the best model
best_model.save('t-100-6500-bilstm.keras')

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step
Best Model Accuracy: 0.9826589595375722
Best Model Precision: 0.9795918367346939
Best Model Recall: 0.9896907216494846
Best Model F1-Score: 0.9846153846153847
Best Model Validation Loss: [0.6757660508155823, 0.5285816788673401, 0.3544233441352844, 0.21315355598926544, 0.13982413709163666, 0.12155063450336456, 0.11289583146572113, 0.10612671077251434, 0.09786263108253479, 0.11741504818201065]
Best Model Validation Accuracy: [0.6589595079421997, 0.9248554706573486, 0.9075144529342651, 0.9653179049491882, 0.9826589822769165, 0.9826589822769165, 0.9826589822769165, 0.9826589822769165, 0.9826589822769165, 0.9710982441902161]


In [222]:
best_model = load_model('t-100-6500-bilstm.keras')

In [23]:
# Read the corpus from the text file
corpus_test = []
with open('corpus_test.txt', 'r', encoding='utf-8') as f:
    corpus_test = [line.strip() for line in f.readlines()]

In [170]:
len(corpus_test)

1000

In [223]:
# Tokenize the corpus
tokenizer = Tokenizer(num_words=6500)  # Use the same vocab size as during training
tokenizer.fit_on_texts(corpus_test)  # Fit tokenizer on the new corpus (could use the same tokenizer as before)
sequences = tokenizer.texts_to_sequences(corpus_test)

# Pad the sequences to ensure the same length as the training data
sentence_length = 171  # Ensure this matches the training sentence length
embedded_docs = pad_sequences(sequences, padding='pre', maxlen=sentence_length)

# Get the true labels (if available) for evaluation
new_y_final = test['kategori_enc'].values  # Assuming 'kategori' column contains labels (0 or 1)

# Predict with the model
new_predictions = (best_model.predict(embedded_docs) > 0.5).astype("int32").flatten()

# Evaluate the new predictions (if labels are available)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

# Print the evaluation results
print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Accuracy on new data: 0.723
Precision on new data: 0.8517350157728707
Recall on new data: 0.54
F1 Score on new data: 0.6609547123623011


In [192]:
voc_size=6500
sentence_length=171

# Convert text to sequences using one-hot encoding and pad the sequences
one_hot_re = [one_hot(sentence, voc_size) for sentence in corpus_test]
new_x_final = pad_sequences(one_hot_re, padding='pre', maxlen=sentence_length)

# Prepare the labels
new_y_final = test['kategori_enc'].values

# Predict with the new data
new_predictions = (best_model.predict(new_x_final) > 0.5).astype("int32").flatten()

# Evaluate the new predictions
new_accuracy = accuracy_score(new_y_final, new_predictions)
new_precision = precision_score(new_y_final, new_predictions)
new_recall = recall_score(new_y_final, new_predictions)
new_f1 = f1_score(new_y_final, new_predictions)

print(f"Accuracy on new data: {new_accuracy}")
print(f"Precision on new data: {new_precision}")
print(f"Recall on new data: {new_recall}")
print(f"F1 Score on new data: {new_f1}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Accuracy on new data: 0.692
Precision on new data: 0.6927710843373494
Recall on new data: 0.69
F1 Score on new data: 0.6913827655310621


### Transformer

In [1]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer

# Load dataset
data = pd.read_excel('dataBerita.xlsx', usecols=['kategori', 'berita'])

# Membersihkan data dengan menghapus whitespace dan karakter tidak perlu
data['kategori'] = data['kategori'].str.strip()        # Menghapus whitespace di awal dan akhir
data['kategori'] = data['kategori'].str.replace(';', '', regex=False)  # Menghapus tanda semicolon
data['kategori_enc'] = data['kategori'].map({'valid': 0, 'hoax': 1})
texts = data['berita'].tolist()

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenisasi dan hitung panjang masing-masing teks
lengths = [len(tokenizer.encode(text)) for text in texts]

# Hitung panjang pada persentil ke-95
percentile_95 = np.percentile(lengths, 95)

# Menampilkan hasil
print(f"Nilai panjang teks pada persentil ke-95: {percentile_95}")


c:\Users\Lenovo\Downloads\Code Thesis\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1424 > 512). Running this sequence through the model will result in indexing errors


Nilai panjang teks pada persentil ke-95: 1031.5499999999997


In [1]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Load dataset
data = pd.read_excel('dataBerita.xlsx', usecols=['kategori', 'berita'])
# Membersihkan data dengan menghapus whitespace dan karakter tidak perlu
data['kategori'] = data['kategori'].str.strip()        # Menghapus whitespace di awal dan akhir
data['kategori'] = data['kategori'].str.replace(';', '', regex=False)  # Menghapus tanda semicolon


c:\Users\Lenovo\Downloads\Code Thesis\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data['kategori_enc'] = data['kategori'].map({'valid': 0, 'hoax': 1})
texts = data['berita'].tolist()
labels = data['kategori_enc'].tolist()

# Initialize RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [3]:
data['kategori_enc'].unique()

array([0, 1])

In [3]:
# Custom Dataset class for RoBERTa
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=216):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
# Define K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_recall = 0
best_model = None
accuracy_scores, precision_scores, recall_scores, f1_scores = [], [], [], []

In [6]:
for fold, (train_index, test_index) in enumerate(kf.split(texts)):
    print(f"Training fold {fold + 1}...")
    
    # Split the data for this fold
    train_texts = [texts[i] for i in train_index]
    test_texts = [texts[i] for i in test_index]
    train_labels = [labels[i] for i in train_index]
    test_labels = [labels[i] for i in test_index]
    
    # Create datasets for this fold
    train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
    test_dataset = CustomDataset(test_texts, test_labels, tokenizer)
    
    # Load model for each fold
    model = RobertaForSequenceClassification.from_pretrained('distilroberta-base', num_labels=2)
    
    # Define training arguments for each fold
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=10,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir='./logs',
        evaluation_strategy="epoch"
    )

    # Define metrics function
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds)
        recall = recall_score(labels, preds)
        f1 = f1_score(labels, preds)
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train and evaluate
    trainer.train()
    eval_results = trainer.evaluate()

    # Extract and store metrics for this fold
    accuracy = eval_results['eval_accuracy']
    precision = eval_results['eval_precision']
    recall = eval_results['eval_recall']
    f1 = eval_results['eval_f1']

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
    # Track the best model based on accuracy
    if recall > best_accuracy:
        best_accuracy = accuracy
        best_model = model

Training fold 1...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Lenovo\Downloads\Code Thesis\env\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18472\1246478591.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 10%|█         | 44/440 [03:28<13:08,  1.99s/it]

{'eval_loss': 0.3351345658302307, 'eval_accuracy': 0.896551724137931, 'eval_precision': 0.8947368421052632, 'eval_recall': 0.8717948717948718, 'eval_f1': 0.8831168831168831, 'eval_runtime': 7.417, 'eval_samples_per_second': 11.73, 'eval_steps_per_second': 1.483, 'epoch': 1.0}


 20%|██        | 88/440 [05:28<12:25,  2.12s/it]

{'eval_loss': 0.4350961148738861, 'eval_accuracy': 0.9080459770114943, 'eval_precision': 0.9428571428571428, 'eval_recall': 0.8461538461538461, 'eval_f1': 0.8918918918918919, 'eval_runtime': 7.7866, 'eval_samples_per_second': 11.173, 'eval_steps_per_second': 1.413, 'epoch': 2.0}


...

 90%|█████████ | 396/440 [20:36<01:36,  2.18s/it]

{'eval_loss': 0.2882876992225647, 'eval_accuracy': 0.9651162790697675, 'eval_precision': 0.9767441860465116, 'eval_recall': 0.9545454545454546, 'eval_f1': 0.9655172413793104, 'eval_runtime': 7.5719, 'eval_samples_per_second': 11.358, 'eval_steps_per_second': 1.453, 'epoch': 9.0}


100%|██████████| 440/440 [22:41<00:00,  3.09s/it]


{'eval_loss': 0.3035212457180023, 'eval_accuracy': 0.9651162790697675, 'eval_precision': 0.9767441860465116, 'eval_recall': 0.9545454545454546, 'eval_f1': 0.9655172413793104, 'eval_runtime': 7.4242, 'eval_samples_per_second': 11.584, 'eval_steps_per_second': 1.482, 'epoch': 10.0}
{'train_runtime': 1361.7328, 'train_samples_per_second': 2.541, 'train_steps_per_second': 0.323, 'train_loss': 0.14948257099498402, 'epoch': 10.0}


100%|██████████| 11/11 [00:06<00:00,  1.58it/s]


In [9]:
# Save the best model
best_model.save_pretrained("transformer")
tokenizer.save_pretrained("transformer")

# Print average metrics across all folds
print(f"Average Accuracy: {np.mean(accuracy_scores)}")
print(f"Average Precision: {np.mean(precision_scores)}")
print(f"Average Recall: {np.mean(recall_scores)}")
print(f"Average F1 Score: {np.mean(f1_scores)}")
print("Best model saved as 'transformer'")

Average Accuracy: 0.9364251982535864
Average Precision: 0.9472070643582272
Average Recall: 0.9230117427791846
Average F1 Score: 0.9346613901595519
Best model saved as 'transformer'


In [10]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

# Load the pre-trained model and tokenizer from the folder
model = RobertaForSequenceClassification.from_pretrained('transformer')
tokenizer = RobertaTokenizer.from_pretrained('transformer')

# Load the new dataset (e.g., 'test.xlsx')
test_data = pd.read_excel('../test.xlsx')

# Extract the true labels for the test set
true_labels = test_data['kategori'].tolist()  # Assuming 'kategori_enc' has the true labels

# Preprocess the new data (tokenize and prepare the input)
test_texts = test_data['berita'].tolist()

# Tokenize the texts
encoding = tokenizer(test_texts, truncation=True, padding=True, max_length=216, return_tensors="pt")

# Get the input tensors
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# Make predictions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation during inference
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits  # Model outputs raw logits

# Convert logits to predicted labels (using argmax to get the highest probability)
predictions = torch.argmax(logits, dim=-1)

# Convert predictions and true labels to numpy arrays
predicted_labels = predictions.numpy()
true_labels = torch.tensor(true_labels).numpy()

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.785
Precision: 0.7567567567567568
Recall: 0.84
F1 Score: 0.7962085308056872
